In [2]:
## ---- Partition Cell Set Network Module Cell Plotting ---- 0682.01.00
## Load Analysis Parameters (Parm1)
## Load Monocle Cell Sets
## Perform Special Gene (gene enrichment) Plots

In [3]:
## Create a Working Input and Output Data Directory, If Id Does Not Exist
parentdir <- '/gpfs/group/torkamani/devans/'
datdir <- paste(parentdir, 'CDC2', sep = '')
if (!file.exists(datdir)) {
    dir.create(datdir)
}
setwd(datdir)

In [4]:
## Read the parameters file
ps <- read.table(file = 'parms.txt', stringsAsFactors = FALSE, header = TRUE)

In [5]:
## Read the GENCODE v27 Data
v27_gen <- read.table(paste(ps$indir, 'v27_Embl_Hugo.txt', sep = ''), header = F)
v27_gen[,1] <- substr(v27_gen[,1] ,1, 15)
v27 <- read.table(paste(ps$indir, 'v27_Embl_Hugo_Type.txt', sep =''))

In [404]:
## Load Monocle3 and Seurat Libraries
library(monocle3)
library(Seurat)
library(dplyr)
library(magrittr)
library(ggplot2)
library(gridExtra)
library(Matrix)
library(rhdf5)
library(grid)
library(igraph)
library(corpcor)
library(fdrtool)
library(SDMTools)

In [271]:
## Read the previously preprocessed downsampled cell set data object
setwd(datdir)
down_stdycds <- readRDS(file = paste(ps$outdir,
            'Aggregated.downsampled.QC.NoDoublets.Repartitioned.rds', sep = ''))

## Build a gene short name to gene id (Ensembl) lookup
short2geneid <- fData(down_stdycds)@rownames
names(short2geneid) <- fData(down_stdycds)@listData$gene_short_name

## Build a gene id (Ensembl) to gene short name lookup
geneid2short <- fData(down_stdycds)@listData$gene_short_name
names(geneid2short) <- fData(down_stdycds)@rownames

In [9]:
## Create variables for how cells sets are organized
cellgrps <- c('healthy', 'diseased', 'healthy', 'diseased', 'healthy', 'diseased')
cellpats <- c('ID Number 1', 'ID Number 1', 'ID Number 2', 'ID Number 2', 'ID Number 3', 'ID Number 3')

## Define and Assign Cell Types
celltypes6 <- c('1-Macrophages',
                '2-Endothelial Cells',
                '3-VSMCs',
                '4-Natural Killer Cells',
                '5-Cytotoxic T Lymphocytes',
                '6-B Lymphocytes')

In [10]:
# Special Genes Set 2:

# ECs:
ec_mod1_genes <- c('ITLN1', 'S100A4', 'SCX')
ec_mod35_genes <- c('ESM1', 'PDGFD') 
ec_mod36_genes <- c('IER2', 'FOS')
## EC Gene IDs
ec_mod1_geneids <- short2geneid[ec_mod1_genes]
ec_mod35_geneids <- short2geneid[ec_mod36_genes]
ec_mod36_geneids <- short2geneid[ec_mod35_genes]

# VSMC:
vsmc_mod9_genes <- c('TNFAIP2', 'NFKBIA', 'SFRP5', 'CRTAC1')
vsmc_mod11_genes <- 'IGSF10'
vsmc_mod13_genes <- c('SPP1', 'IBSP', 'POSTN')
vsmc_mod30_genes <- c('TRA3', 'SEMA3C', 'C1QTNF3', 'NTM', 'FBN1')
vsmc_mod31_genes <- c('ZFP36', 'EGR1', 'JUNB', 'FOSB', 'FOS', 'IER2')
vsmc_mod31plus2_genes <- c(vsmc_mod31_genes, 'TNFAIP2', 'NFKBIA')
vsmc_mod9.31_genes <- c(vsmc_mod9_genes, vsmc_mod31_genes)
## VSMC Gene IDs
vsmc_mod13_geneids <- short2geneid[vsmc_mod9_genes]
vsmc_mod11_geneids <- short2geneid[vsmc_mod11_genes]
vsmc_mod13_geneids <- short2geneid[vsmc_mod13_genes]
vsmc_mod30_geneids <- short2geneid[vsmc_mod30_genes]
vsmc_mod31_geneids <- short2geneid[vsmc_mod31_genes]
vsmc_mod9.31_geneids <- short2geneid[vsmc_mod9.31_genes]
vsmc_mod31plus2_geneids <- short2geneid[vsmc_mod31plus2_genes]

## EC Module Special Gene Plots, Modules 1 and 36

In [11]:
scale <- 2
cs <- .7 * scale
alpha <- .5
p <- 2
path <- paste(datdir, '/', ps$outdir, celltypes6[p], sep = '')
plotfile1 <- paste(path, '/CellPlot.GeneEnrichment.EC.Module1.SpecialGeneSet2.rev2.pdf', sep = '') 
plotfile2 <- paste(path, '/CellPlot.GeneEnrichment.EC.Module36.SpecialGeneSet2.rev2.pdf', sep = '')
length(ec_mod1_genes)
length(ec_mod36_genes)

pdf(plotfile1, width = 9 * 1.5 * scale, height = 7 * scale)
plot_cells(down_stdycds[, colData(down_stdycds)$assigned_cell_type == celltypes6[p]],
           genes = ec_mod1_genes, label_cell_groups=FALSE, cell_size = cs, alpha = alpha)
dev.off()

pdf(plotfile2, width = 7 * 1.5 * scale, height = 7 * scale)
plot_cells(down_stdycds[, colData(down_stdycds)$assigned_cell_type == celltypes6[p]],
           genes = ec_mod36_genes, label_cell_groups=FALSE, cell_size = cs, alpha = alpha)
dev.off()

[1] 3

[1] 2

No trajectory to plot. Has learn_graph() been called yet?


png 
  2

No trajectory to plot. Has learn_graph() been called yet?


png 
  2

## VSMC Module Special Gene Plots, Modules 13 and 31

In [95]:
scale <- 2
cs <- .7 * scale
alpha <- .5
p <- 3
path <- paste(datdir, '/', ps$outdir, celltypes6[p], sep = '')
plotfile1 <- paste(path, '/CellPlot.GeneEnrichment.VSMC.Module13.SpecialGeneSet2.rev2.pdf', sep = '') 
plotfile2 <- paste(path, '/CellPlot.GeneEnrichment.VSMC.Module31.SpecialGeneSet2.rev2.pdf', sep = '')
length(vsmc_mod13_genes)
length(vsmc_mod31_genes)

pdf(plotfile1, width = 9 * 1.5 * scale, height = 7 * scale)
    plot_cells(down_stdycds[, colData(down_stdycds)$assigned_cell_type == celltypes6[p]],
           genes = vsmc_mod13_genes, label_cell_groups=FALSE, cell_size = cs, alpha = alpha)
dev.off()

pdf(plotfile2, width = 9 * 1.5 * scale, height = 12 * scale)
plot_cells(down_stdycds[, colData(down_stdycds)$assigned_cell_type == celltypes6[p]],
           genes = vsmc_mod31_genes, label_cell_groups=FALSE, cell_size = cs, alpha = alpha)
dev.off()

[1] 3

[1] 6

No trajectory to plot. Has learn_graph() been called yet?


png 
  2

No trajectory to plot. Has learn_graph() been called yet?


png 
  2

In [16]:
Macrophages <- c('APOC1', 'FABP5', 'APOE', 'SDS', 'LYVE1', 'SCN9A', 'S100A12', 'P2RY14')
T_Cells     <- c('IGKC', 'MGP', 'IGLC2', 'PTGDS', 'FGFBP2', 'KLRF1')
B_Cells     <- c('MGP', 'TXNIP', 'HERPUD1', 'ZNF665', 'CRIP1', 'RAB11FIP1')
NKT_cells   <- c('IGKC', 'DUSP1', 'ZFP36', 'TSC22D3')

# Other VSMC and EC Genes for Enrichment plots

VSMC_AC_Cells <- c('SPP1', 'SFRP5', 'IBSP', 'CRTAC1')
VSMC_PA_Cells <- c('APOD', 'PLA2G2A', 'C3', 'MFAP5')
EC_AC_Cells <- c('ITLN1', 'DKK2', 'F5', 'FN1')
EC_PA_Cells <- c('IL6', 'MLPH', 'HLA-DQA1', 'ACKR1')

## Declare Tom's best genes for definiting cell types
toms_markers7 <- c('AIF1', 'CD14', 'CD68',
                   'VWF', 'PECAM1', 'ECSCR',
                   'CALD1', 'MYL9', 'TAGLN',
                   'NKG7', 'XCL1', 'CTSW',
                   'CD2', 'TRAC', 'CD69',
                   'CD79A', 'MS4A1', 'IGKC')

toms_gene_ids7 <- short2geneid[toms_markers7]

## Macrophages Special Gene Plots

In [117]:
scale <- .5
cs <- .7 * scale
alpha <- .5
p <- 1
path <- paste(datdir, '/', ps$outdir, celltypes6[p], sep = '')
plotfile1 <- paste(path, '/CellPlot.GeneEnrichment.Macrophages.SpecialGeneSet2.rev2.pdf', sep = '') 

pdf(plotfile1, width = 10 * 1.5 * scale, height = 15 * scale)
    plot_cells(down_stdycds[, colData(down_stdycds)$assigned_cell_type == celltypes6[p]],
           genes = Macrophages, label_cell_groups=FALSE, cell_size = cs, alpha = alpha)
dev.off()

No trajectory to plot. Has learn_graph() been called yet?


png 
  2

## T_Cells Special Gene Plots

In [115]:
scale <- .5
cs <- .7 * scale
alpha <- .5
p <- 5
path <- paste(datdir, '/', ps$outdir, celltypes6[p], sep = '')
plotfile1 <- paste(path, '/CellPlot.GeneEnrichment.T_Cells.SpecialGeneSet2.rev2.pdf', sep = '') 

pdf(plotfile1, width = 10 * 1.5 * scale, height = 12 * scale)
    plot_cells(down_stdycds[, colData(down_stdycds)$assigned_cell_type == celltypes6[p]],
           genes = T_Cells, label_cell_groups=FALSE, cell_size = cs, alpha = alpha)
dev.off()

No trajectory to plot. Has learn_graph() been called yet?


png 
  2

## B_Cells Special Gene Plots

In [120]:
scale <- .5
cs <- .7 * scale
alpha <- .5
p <- 6
path <- paste(datdir, '/', ps$outdir, celltypes6[p], sep = '')
plotfile1 <- paste(path, '/CellPlot.GeneEnrichment.B_Cells.SpecialGeneSet2.rev2.pdf', sep = '') 

pdf(plotfile1, width = 10 * 1.5 * scale, height = 10 * scale)
    plot_cells(down_stdycds[, colData(down_stdycds)$assigned_cell_type == celltypes6[p]],
           genes = B_Cells, label_cell_groups=FALSE, cell_size = cs, alpha = alpha)
dev.off()

No trajectory to plot. Has learn_graph() been called yet?


png 
  2

## NKT_cells Special Gene Plots

In [122]:
scale <- .5
cs <- .7 * scale * 3
alpha <- .5
p <- 4
path <- paste(datdir, '/', ps$outdir, celltypes6[p], sep = '')
plotfile1 <- paste(path, '/CellPlot.GeneEnrichment.NKT_cells.SpecialGeneSet2.rev2.pdf', sep = '') 

pdf(plotfile1, width = 10 * 1.5 * scale, height = 10 * scale)
    plot_cells(down_stdycds[, colData(down_stdycds)$assigned_cell_type == celltypes6[p]],
           genes = NKT_cells, label_cell_groups=FALSE, cell_size = cs, alpha = alpha)
dev.off()

No trajectory to plot. Has learn_graph() been called yet?


png 
  2

## All Partition Marker Special Gene Plots

In [140]:
scale <- 1
cs <- .7 * scale * .2
alpha <- .5
path <- paste(datdir, '/', ps$outdir, sep = '')
plotfile1 <- paste(path, '/CellPlot.GeneEnrichment.PartitionMarkers7.SpecialGeneSet2.rev2.pdf', sep = '') 

pdf(plotfile1, width = 10 * 1.5 * scale, height = 12 * scale)
    plot_cells(down_stdycds, genes = toms_markers7, label_cell_groups = FALSE, cell_size = cs, alpha = alpha)
dev.off()

No trajectory to plot. Has learn_graph() been called yet?


png 
  2

## VSMC AC Special Genes Plot

In [26]:
scale <- .5
cs <- .7 * scale * 1
alpha <- .5
p <- 3
path <- paste(datdir, '/', ps$outdir, celltypes6[p], sep = '')
plotfile1 <- paste(path, '/CellPlot.GeneEnrichment.VSMC.AC.SpecialGeneSet3.rev1.pdf', sep = '') 

pdf(plotfile1, width = 10 * 1.5 * scale, height = 10 * scale)
    plot_cells(down_stdycds[, colData(down_stdycds)$assigned_cell_type == celltypes6[p]],
           genes = VSMC_AC_Cells, label_cell_groups=FALSE, cell_size = cs, alpha = alpha)
dev.off()

No trajectory to plot. Has learn_graph() been called yet?


png 
  2

## VSMC PA Special Genes Plot

In [27]:
scale <- .5
cs <- .7 * scale * 1
alpha <- .5
p <- 3
path <- paste(datdir, '/', ps$outdir, celltypes6[p], sep = '')
plotfile1 <- paste(path, '/CellPlot.GeneEnrichment.VSMC.PA.SpecialGeneSet3.rev1.pdf', sep = '') 

pdf(plotfile1, width = 10 * 1.5 * scale, height = 10 * scale)
    plot_cells(down_stdycds[, colData(down_stdycds)$assigned_cell_type == celltypes6[p]],
           genes = VSMC_PA_Cells, label_cell_groups=FALSE, cell_size = cs, alpha = alpha)
dev.off()

No trajectory to plot. Has learn_graph() been called yet?


png 
  2

## EC AC Special Genes Plot

In [29]:
scale <- .5
cs <- .7 * scale * 1
alpha <- .5
p <- 2
path <- paste(datdir, '/', ps$outdir, celltypes6[p], sep = '')
plotfile1 <- paste(path, '/CellPlot.GeneEnrichment.EC.AC.SpecialGeneSet3.rev1.pdf', sep = '') 

pdf(plotfile1, width = 10 * 1.5 * scale, height = 10 * scale)
    plot_cells(down_stdycds[, colData(down_stdycds)$assigned_cell_type == celltypes6[p]],
           genes = EC_AC_Cells, label_cell_groups=FALSE, cell_size = cs, alpha = alpha)
dev.off()

No trajectory to plot. Has learn_graph() been called yet?


png 
  2

## EC PA Special Genes Plot

In [93]:
scale <- .5
cs <- .7 * scale * 1
alpha <- .5
p <- 2
path <- paste(datdir, '/', ps$outdir, celltypes6[p], sep = '')
plotfile1 <- paste(path, '/CellPlot.GeneEnrichment.EC.PA.SpecialGeneSet3.rev1.pdf', sep = '') 

pdf(plotfile1, width = 10 * 1.5 * scale, height = 10 * scale)
    plot_cells(down_stdycds[, colData(down_stdycds)$assigned_cell_type == celltypes6[p]],
           genes = EC_PA_Cells, label_cell_groups=FALSE, cell_size = cs, alpha = alpha)
dev.off()

No trajectory to plot. Has learn_graph() been called yet?


png 
  2